In [48]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

datafile=pd.read_csv('/content/letter-recognition.csv')
diagram_map = {'A': 1.0, 'B': -1.0,'C': 1.0, 'D': -1.0,'E': 1.0, 'F': -1.0,'G': 1.0, 'H': -1.0,'I': 1.0, 'J': -1.0,'K': 1.0, 'L': -1.0,'M': 1.0, 'N': -1.0
            ,'O': 1.0, 'P': -1.0,'Q': 1.0, 'R': -1.0,'S': 1.0, 'T': -1.0,'U': 1.0, 'V': -1.0,'W': 1.0, 'X': -1.0,
            'Y': 1.0, 'Z': -1.0}
datafile['diagnosis'] = datafile['diagnosis'].map(diagram_map)
datafile

X=datafile.drop('diagnosis',axis=1)
y=datafile['diagnosis']
m=8000
X=X[:m]
y=y[:m]
print(X.shape)

datafile1=datafile
datafile1=datafile1.sample(frac=1)
X1=datafile1.drop('diagnosis',axis=1)[:m]
y1=datafile1['diagnosis'][:m]

num=int(0.8*m)
X_train=X1[:num]
y_train=y1[:num]
X_test=X1[num:]
y_test=y1[num:]
print(X_train.shape,X_test.shape)


(8000, 16)
(6400, 16) (1600, 16)


**RESULTS SVM WITHOUT MARKOV SAMPLING**

In [49]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, y_pred)))

Num1=1000
X_val=X[:Num1]
y_val=y[:Num1]
print(y_val.shape)

from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_val, y_val)

y_pre=clf.predict(X_test)
print(y_pre)


accuracy on test dataset: 0.734375
recall on test dataset: 0.7630573248407644
precision on test dataset: 0.7630573248407644
(1000,)
[-1. -1. -1. ...  1.  1. -1.]


In [50]:
import math

def Cost(Y_pred, Y):
    # calculate hinge loss
    if Y_pred*Y>1.0:
        return 0
    return 1.0-Y_pred*Y

X=X.to_numpy()
y=y.to_numpy()
Ni=0
j=0
k=2
q=1/1.2
m=y.shape[0]

mp=0
mn=0
ct=0
X_tr=[]
y_tr=[]
X_te=[]
y_te=[]
X_t=X[0].reshape(1,-1)
print(X_t)
y_t=y[0].reshape(1,-1)[0]
X_tr.append(X_t[0])
y_tr.append(y_t[0])

if m%2==0 and y_t[0]==1.0:
    mp+=1
elif m%2==0 and y_t[0]==-1.0:
    mn+=1
pred_t=clf.predict(X_t)

p=1999999.0
for i in range(1,m):
    X_z=X[i].reshape(1,-1)
    y_z=y[i].reshape(1,-1)[0]
    pred_z=clf.predict(X_z)
    P=math.exp(-Cost(pred_z[0],y_z[0]))/math.exp(-Cost(pred_t[0],y_t[0]))
    if ct>=k:
        Pdd=q*P
        if Pdd>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z[0])
            ct+=1 
            X_t=X_z
            y_t=y_z
            pred_t=pred_z
            Ni+=1
        else:
            X_te.append(X_z[0])
            y_te.append(y_z[0])
            ct=0
    elif P==1.0 and y_z==y_t:
        Pd=math.exp(-y_z[0]*pred_z[0])/math.exp(-y_t[0]*pred_t[0])
        if Pd>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z[0])
            ct+=1
            X_t=X_z
            y_t=y_z
            pred_t=pred_z
            Ni+=1
        else:
            X_te.append(X_z[0])
            y_te.append(y_z[0])
            ct=0
    elif P==1.0 and y_z[0]*y_t[0]==-1.0 or P<1.0:
        Pd=math.exp(-y_z[0]*pred_z[0])/math.exp(-y_t[0]*pred_t[0])
        if Pd>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z[0])
            ct+=1
            X_t=X_z
            y_t=y_z
            pred_t=pred_z
            Ni+=1
        else:
            X_te.append(X_z[0])
            y_te.append(y_z[0])
            ct=0
    elif P>1.0:
        X_tr.append(X_z[0])
        y_tr.append(y_z[0])
        ct+=1
        X_t=X_z
        y_t=y_z
        pred_t=pred_z
        Ni+=1
    if Ni>=N1:
        clf.fit(np.array(X_tr),np.array(y_tr))
        j+=1
    if j>=k:
        break

[[ 2  8  3  5  1  8 13  0  6  6 10  8  0  8  0  8]]


In [51]:
X_tr=np.array(X_tr)
y_tr=np.array(y_tr)
X_te=np.array(X_te)
y_te=np.array(y_te)
clf = svm.SVC(kernel='linear')
clf.fit(X_tr, y_tr)

y_pred=clf.predict(X_test)

print("Accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("Recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("Precision on test dataset: {}".format(recall_score(y_test, y_pred)))

Accuracy on test dataset: 0.73125
Recall on test dataset: 0.7006369426751592
Precision on test dataset: 0.7006369426751592
